In [2]:
pip install autogluon

     |████████████████████████████████| 352 kB 32.0 MB/s 
     |████████████████████████████████| 56 kB 4.3 MB/s 
     |████████████████████████████████| 52 kB 1.4 MB/s 
     |████████████████████████████████| 273 kB 53.1 MB/s 
     |████████████████████████████████| 27.4 MB 52.4 MB/s 
     |████████████████████████████████| 210 kB 69.6 MB/s 
     |████████████████████████████████| 131 kB 73.5 MB/s 
     |████████████████████████████████| 22.3 MB 1.4 MB/s 
     |████████████████████████████████| 802 kB 45.7 MB/s 
     |████████████████████████████████| 4.2 MB 34.7 MB/s 
     |████████████████████████████████| 1.3 MB 46.7 MB/s 
     |████████████████████████████████| 119 kB 59.8 MB/s 
     |████████████████████████████████| 3.0 MB 53.3 MB/s 
     |████████████████████████████████| 296 kB 59.2 MB/s 
     |████████████████████████████████| 189 kB 61.2 MB/s 
     |████████████████████████████████| 2.0 MB 41.7 MB/s 
     |████████████████████████████████| 166.7 MB 15 kB/s 
     |███████████

In [3]:
pip install imodels

     |████████████████████████████████| 109 kB 25.9 MB/s 
     |████████████████████████████████| 1.3 MB 34.4 MB/s 
  Attempting uninstall: mlxtend
    Found existing installation: mlxtend 0.14.0
    Uninstalling mlxtend-0.14.0:
      Successfully uninstalled mlxtend-0.14.0


In [4]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor

In [5]:
# train data
train_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv')
subsample_size = 100  # subsample subset of data for faster demo, try setting this to much larger values
train_data = train_data.sample(n=subsample_size, random_state=0)
# train_data.head()

# test data
test_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv')
label = 'class'
y_test = test_data[label]  # values to predict
test_data_nolab = test_data.drop(columns=[label])  # delete label column to prove we're not cheating
# test_data_nolab.head()
# print("Summary of class variable: \n", train_data[label].describe())

In [6]:
predictor = TabularPredictor(label=label).fit(train_data, time_limit=8, verbosity=2)

No path specified. Models will be saved in: "AutogluonModels/ag-20220102_152026/"
Beginning AutoGluon training ... Time limit = 8s
AutoGluon will save models to "AutogluonModels/ag-20220102_152026/"
AutoGluon Version:  0.3.1
Train Data Rows:    100
Train Data Columns: 14
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [' >50K', ' <=50K']
	If 'binary' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
NumExpr defaulting to 2 threads.
Selected class <--> label mapping:  class 1 =  >50K, class 0 =  <=50K
	Note: For your binary classification, AutoGluon arbitrarily selected which label-value represents positive ( >50K) vs negative ( <=50K) class.
	To explicitly set the positive_class, either rename classes to 1 and 0, or specify positive_class in Predictor init.
Using 

In [7]:
predictor.leaderboard(silent=True)

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost,0.85,0.015986,1.713479,0.015986,1.713479,1,True,7
1,WeightedEnsemble_L2,0.85,0.016931,2.245261,0.000945,0.531782,2,True,10
2,LightGBM,0.80,0.013686,0.167239,0.013686,0.167239,1,True,4
3,LightGBMXT,0.80,0.013996,0.638525,0.013996,0.638525,1,True,3
4,ExtraTreesEntr,0.80,0.237644,1.059356,0.237644,1.059356,1,True,9
5,ExtraTreesGini,0.75,0.241263,1.194778,0.241263,1.194778,1,True,8
6,KNeighborsUnif,0.70,0.104408,0.012150,0.104408,0.012150,1,True,1
7,RandomForestGini,0.70,0.107785,0.690611,0.107785,0.690611,1,True,5
8,RandomForestEntr,0.70,0.116520,0.777813,0.116520,0.777813,1,True,6
9,KNeighborsDist,0.65,0.104032,0.009760,0.104032,0.009760,1,True,2


In [15]:
predictor.feature_importance(train_data)

Computing feature importance via permutation shuffling for 14 features using 100 rows with 3 shuffle sets...
	4.8s	= Expected runtime (1.6s per shuffle set)
	0.29s	= Actual runtime (Completed 3 of 3 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
age,0.123333,0.040415,0.016989,3,0.354913,-0.108246
relationship,0.093333,0.015275,0.004405,3,0.180862,0.005804
hours-per-week,0.036667,0.005774,0.004082,3,0.069749,0.003584
sex,0.036667,0.005774,0.004082,3,0.069749,0.003584
education-num,0.033333,0.011547,0.018875,3,0.099499,-0.032832
marital-status,0.023333,0.015275,0.059041,3,0.110862,-0.064196
native-country,0.016667,0.011547,0.064806,3,0.082832,-0.049499
capital-loss,0.000000,0.000000,0.500000,3,0.000000,0.000000
capital-gain,0.000000,0.000000,0.500000,3,0.000000,0.000000
race,0.000000,0.000000,0.500000,3,0.000000,0.000000


In [17]:
predictor.get_model_best()

'CatBoost'

In [20]:
predictor.save()

TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20220102_152026/")
